In [1]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,}'.format
import seaborn as sns

### Reading a File

In [2]:
# Create data path variable for loading data
data_path = '/Users/yangweichle/Documents/Employment/TRAINING/DATA SCIENCE/GCP BigQuery API_Python/Datasets/'

In [3]:
# Read file into DataFrame
# Note: header=0 disables the header from the file
#       dtype: type name or dict of column -> type, default None; data type for data or columns;
#              e.g. {'a': np.float64, 'b': np.int32}; use 'str' or 'object' together with suitable 'na_values' settings
#              to preserve and not interpret dtype
chicago_crime = pd.read_csv(data_path + 'chicago_crime.csv', header=0, 
                            dtype={'iucr': 'object', 'fbi_code': 'object'}) # Comma-separated values file

### Examine the DataFrame

In [4]:
# Return the number of rows and columns (dimensionality) of the DataFrame
print('Rows: {}, Cols: {}'.format(chicago_crime.shape[0], chicago_crime.shape[1]))

Rows: 6876354, Cols: 22


In [5]:
# Print a concise summary of a DataFrame including the index dtype and column dtypes, non-null values, and memory usage
# Note: Useful to quickly see if null values exist 
chicago_crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6876354 entries, 0 to 6876353
Data columns (total 22 columns):
unique_key              int64
case_number             object
date                    object
block                   object
iucr                    object
primary_type            object
description             object
location_description    object
arrest                  bool
domestic                bool
beat                    int64
district                float64
ward                    float64
community_area          float64
fbi_code                object
x_coordinate            float64
y_coordinate            float64
year                    int64
updated_on              object
latitude                float64
longitude               float64
location                object
dtypes: bool(2), float64(7), int64(3), object(10)
memory usage: 1.0+ GB


In [6]:
# Column names (which is "an index")
chicago_crime.columns

Index(['unique_key', 'case_number', 'date', 'block', 'iucr', 'primary_type',
       'description', 'location_description', 'arrest', 'domestic', 'beat',
       'district', 'ward', 'community_area', 'fbi_code', 'x_coordinate',
       'y_coordinate', 'year', 'updated_on', 'latitude', 'longitude',
       'location'],
      dtype='object')

#### Identifying important columns to include in the DataFrame.

In [7]:
# Select multiple columns
include_cols = ['date', 'block', 'iucr', 'primary_type', 'description', 'location_description', 'arrest', 'domestic',
                'beat', 'district', 'ward', 'community_area', 'fbi_code', 'x_coordinate', 'y_coordinate', 'year',
                'updated_on', 'latitude', 'longitude', 'location'] 
chicago_crime = chicago_crime[include_cols]

In [8]:
# Return the number of rows and columns (dimensionality) of the DataFrame
print('Rows: {}, Cols: {}'.format(chicago_crime.shape[0], chicago_crime.shape[1]))

Rows: 6876354, Cols: 20


In [9]:
# Rename one or more columns in the original DataFrame rather than returning a new view
chicago_crime = chicago_crime.rename(columns={'location': 'lat_long'})

In [10]:
# Print a concise summary of a DataFrame including the index dtype and column dtypes, non-null values, and memory usage
# Note: Useful to quickly see if null values exist 
chicago_crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6876354 entries, 0 to 6876353
Data columns (total 20 columns):
date                    object
block                   object
iucr                    object
primary_type            object
description             object
location_description    object
arrest                  bool
domestic                bool
beat                    int64
district                float64
ward                    float64
community_area          float64
fbi_code                object
x_coordinate            float64
y_coordinate            float64
year                    int64
updated_on              object
latitude                float64
longitude               float64
lat_long                object
dtypes: bool(2), float64(7), int64(2), object(9)
memory usage: 957.4+ MB


### Examine the Columns

#### Drop non-criminal observations

In [11]:
# Return elements, either from `x` or `y`, depending on `condition`
chicago_crime['non_criminal'] = np.where(chicago_crime.primary_type.isin(['NON-CRIMINAL', 'NON - CRIMINAL', 'NON-CRIMINAL (SUBJECT SPECIFIED)']), 1, 0)

In [12]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
chicago_crime.non_criminal.value_counts(dropna=False)

0    6876137
1        217
Name: non_criminal, dtype: int64

In [13]:
# Excluding non-criminal values
chicago_crime = chicago_crime[chicago_crime.non_criminal == 0]
chicago_crime.drop('non_criminal', axis=1, inplace=True)

In [14]:
# Return the number of rows and columns (dimensionality) of the DataFrame
print('Rows: {}, Cols: {}'.format(chicago_crime.shape[0], chicago_crime.shape[1]))

Rows: 6876137, Cols: 20


#### Combining 'OTHER NARCOTIC VIOLATION' with 'NARCOTICS'

In [15]:
# Replace values in a column
# Note: inplace=True changes the original DataFrame
chicago_crime.primary_type.replace('OTHER NARCOTIC VIOLATION', 'NARCOTICS', inplace=True)

#### Combining 'DOMESTIC VIOLENCE' with 'ASSAULT'

In [16]:
# Replace values in a column
# Note: inplace=True changes the original DataFrame
chicago_crime.primary_type.replace('DOMESTIC VIOLENCE', 'ASSAULT', inplace=True)

#### Create indicators for Index/Non-Index offense based on CSV file: [Chicago Police Department - Illinois Uniform Crime Reporting (IUCR) Codes](https://data.cityofchicago.org/Public-Safety/Chicago-Police-Department-Illinois-Uniform-Crime-R/c7ck-438e).

In [17]:
# Return elements, either from `x` or `y`, depending on `condition`
chicago_crime['index_offense'] = np.where(chicago_crime.iucr.isin(['0110', '0130', '0261', '0262', '0263', '0264',
                                                                   '0265', '0266', '0271', '0272', '0273', '0274',
                                                                   '0275', '0281', '0291', '0312', '0313', '031A',
                                                                   '031B', '0320', '0325', '0326', '0330', '0331',
                                                                   '0334', '0337', '033A', '033B', '0340', '041A',
                                                                   '041B', '0420', '0430', '0450', '0451', '0452',
                                                                   '0453', '0461', '0462', '0479', '0480', '0481',
                                                                   '0482', '0483', '0485', '0487', '0488', '0489',
                                                                   '0490', '0491', '0492', '0493', '0495', '0496',
                                                                   '0497', '0498', '0510', '051A', '051B', '0520',
                                                                   '0530', '0550', '0551', '0552', '0553', '0555',
                                                                   '0556', '0557', '0558', '0610', '0620', '0630',
                                                                   '0650', '0810', '0820', '0850', '0860', '0865',
                                                                   '0870', '0880', '0890', '0895', '0910', '0915',
                                                                   '0917', '0918', '0920', '0925', '0927', '0928',
                                                                   '0930', '0935', '0937', '0938', '1010', '1020',
                                                                   '1025', '1090', '1753', '1754']), 1, 0)

In [18]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
print(chicago_crime.index_offense.value_counts(dropna=False))
round(chicago_crime.index_offense.value_counts(dropna=False) / len(chicago_crime), 3)

0    4177733
1    2698404
Name: index_offense, dtype: int64


0   0.608
1   0.392
Name: index_offense, dtype: float64

In [19]:
# Return elements, either from `x` or `y`, depending on `condition`
chicago_crime['nonindex_offense'] = np.where(chicago_crime.iucr.isin(['0141', '0142', '0440', '0454', '0460', '0470',
                                                                      '0475', '0484', '0486', '0494', '0545', '0554', 
                                                                      '0560', '0580', '0581', '0583', '0584', '1030',
                                                                      '1035', '1050', '1055', '1110', '1120', '1121',
                                                                      '1122', '1130', '1135', '1140', '1150', '1151',
                                                                      '1152', '1153', '1154', '1155', '1156', '1160',
                                                                      '1170', '1185', '1195', '1200', '1205', '1206',
                                                                      '1210', '1220', '1230', '1235', '1240', '1241',
                                                                      '1242', '1245', '1255', '1260', '1624', '1261',
                                                                      '1265', '1305', '1310', '1320', '1330', '1335',
                                                                      '1340', '1345', '1350', '1360', '1365', '1370',
                                                                      '1375', '141A', '141B', '141C', '142A', '142B',
                                                                      '1435', '143A', '143B', '143C', '1440', '1450',
                                                                      '1460', '1475', '1476', '1477', '1478', '1479',
                                                                      '1480', '1481', '1505', '1506', '1507', '1510',
                                                                      '1511', '1512', '1513', '1515', '1520', '1521',
                                                                      '1525', '1526', '1530', '1531', '1535', '1536',
                                                                      '1537', '1540', '1541', '1542', '1544', '1549',
                                                                      '1562', '1563', '1564', '1565', '1566', '1570',
                                                                      '1572', '1574', '1576', '1578', '1580', '1582',
                                                                      '1585', '1590', '1610', '1611', '1620', '1621',
                                                                      '1622', '1623', '1625', '1626', '1627', '1630',
                                                                      '1631', '1632', '1633', '1640', '1650', '1651',
                                                                      '1661', '1670', '1680', '1681', '1682', '1690',
                                                                      '1691', '1692', '1693', '1694', '1695', '1696',
                                                                      '1697', '1710', '1715', '1720', '1725', '1750',
                                                                      '1751', '1752', '1755', '1775', '1780', '1790',
                                                                      '1791', '1792', '1811', '1812', '1821', '1822',
                                                                      '1840', '1850', '1860', '1900', '2010', '2011',
                                                                      '2012', '2013', '2014', '2015', '2016', '2017',
                                                                      '2018', '2019', '2020', '2021', '2022', '2023',
                                                                      '2024', '2025', '2026', '2027', '2028', '2029',
                                                                      '2030', '2031', '2032', '2033', '2034', '2040',
                                                                      '2050', '2060', '2070', '2080', '2090', '2091',
                                                                      '2092', '2093', '2094', '2095', '2110', '2111',
                                                                      '2120', '2160', '2170', '2210', '2220', '2230',
                                                                      '2240', '2250', '2251', '2500', '2820', '2825',
                                                                      '2826', '2830', '2840', '2850', '2851', '2860',
                                                                      '2870', '2890', '2895', '2900', '3000', '3100',
                                                                      '3200', '3300', '3400', '3610', '3710', '3720',
                                                                      '3730', '3731', '3740', '3750', '3751', '3760',
                                                                      '3770', '3800', '3910', '3920', '3960', '3966',
                                                                      '3970', '3975', '3980', '4210', '4220', '4230',
                                                                      '4240', '4255', '4310', '4386', '4387', '4388',
                                                                      '4389', '4410', '4420', '4510', '4625', '4650',
                                                                      '4651', '4652', '4740', '4750', '4800', '4810',
                                                                      '4860', '5000', '5001', '5002', '5003', '5004',
                                                                      '5007', '5009', '500E', '500N', '5011', '501A',
                                                                      '501H', '502P', '502R', '502T', '5110', '5111',
                                                                      '5112', '5120', '5121', '5122', '5130', '5131',
                                                                      '5132']), 1, 0)

In [20]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
print(chicago_crime.nonindex_offense.value_counts(dropna=False))
round(chicago_crime.nonindex_offense.value_counts(dropna=False) / len(chicago_crime), 3)

1    4110316
0    2765821
Name: nonindex_offense, dtype: int64


1   0.598
0   0.402
Name: nonindex_offense, dtype: float64

In [21]:
# Compute a frequency table of two factors
# Builtin-Styles - https://pandas.pydata.org/pandas-docs/stable/style.html
orange = sns.light_palette('orange', as_cmap=True)
pd.crosstab(chicago_crime.index_offense.fillna(' NaN'), chicago_crime.nonindex_offense.fillna(' NaN'), margins=True, margins_name='Total', dropna=False).style.background_gradient(cmap=orange)

nonindex_offense,0,1,Total
index_offense,,,
0,67417,4110316,4177733
1,2698404,0,2698404
Total,2765821,4110316,6876137


Note: There are 67,417 records that are not consistent with the Index/Non-Index offense indicators as defined.

The following `iucr` codes are not contained in the CSV files as described above.

In [22]:
chicago_crime[(chicago_crime.index_offense == 0) & (chicago_crime.nonindex_offense == 0)].iucr.value_counts(dropna=False)

0840    44531
0841    15206
0843     2955
0842     2901
0499     1546
5008      189
1581       72
5005        6
5013        5
3961        4
0830        1
9901        1
Name: iucr, dtype: int64

In [23]:
# Return DataFrame with duplicate rows removed, optionally only considering certain columns
chicago_crime[['iucr', 'index_offense', 'nonindex_offense', 'primary_type', 'description']][(chicago_crime.index_offense == 0) & (chicago_crime.nonindex_offense == 0)].drop_duplicates().sort_values(by='iucr')

,iucr,index_offense,nonindex_offense,primary_type,description
11793,0499,0,0,BATTERY,AGGRAVATED DOMESTIC BATTERY
414873,0830,0,0,THEFT,THEFT RETAIL
73281,0840,0,0,THEFT,FINANCIAL ID THEFT: OVER $300
2482,0841,0,0,THEFT,FINANCIAL ID THEFT:$300 &UNDER
5426,0842,0,0,THEFT,AGG: FINANCIAL ID THEFT
632,0843,0,0,THEFT,ATTEMPT FINANCIAL IDENTITY THEFT
7559,1581,0,0,SEX OFFENSE,NON-CONSENSUAL DISSEMINATION PRIVATE SEXUAL IM...
1719493,3961,0,0,INTIMIDATION,INTIMIDATION OF LAW ENFORCEMENT OFFICIAL
625532,5005,0,0,SEX OFFENSE,ATT AGG CRIM SEXUAL ABUSE
5444,5008,0,0,OTHER OFFENSE,FIREARM REGISTRATION VIOLATION


The [Chicago Police Department CLEARMAP Crime Summary](http://gis.chicagopolice.org/clearmap_crime_sums/crime_types.html#N04A) webpage was referenced to see whether the above IUCR codes could be categorized into Index/Non-Index indicators.

In [24]:
# Return elements, either from `x` or `y`, depending on `condition`
chicago_crime['iucr_index_offense_update'] = np.where(chicago_crime.iucr.isin(['0499', '0830', '0840', '0841', '0842', '0843', '9901']), 1, 0)

In [25]:
chicago_crime.iucr_index_offense_update.value_counts()

0    6808996
1      67141
Name: iucr_index_offense_update, dtype: int64

In [26]:
# Return elements, either from `x` or `y`, depending on `condition`
chicago_crime['iucr_nonindex_offense_update'] = np.where(chicago_crime.iucr.isin(['1581', '3961', '5005', '5008', '5013']), 1, 0)

In [27]:
chicago_crime.iucr_nonindex_offense_update.value_counts()

0    6875861
1        276
Name: iucr_nonindex_offense_update, dtype: int64

`index_offense` values will be updated for records that were flagged by `iucr_index_offense_update`.

In [28]:
# Return elements, either from `x` or `y`, depending on `condition`
chicago_crime['index_offense'] = np.where(chicago_crime.iucr_index_offense_update == 1, 1, chicago_crime.index_offense)
chicago_crime.drop('iucr_index_offense_update', axis=1, inplace=True)

`nonindex_offense` values will be updated for records that were flagged by `iucr_nonindex_offense_update`.

In [29]:
# Return elements, either from `x` or `y`, depending on `condition`
chicago_crime['nonindex_offense'] = np.where(chicago_crime.iucr_nonindex_offense_update == 1, 1, chicago_crime.nonindex_offense)
chicago_crime.drop('iucr_nonindex_offense_update', axis=1, inplace=True)

In [30]:
# Compute a frequency table of two factors
# Builtin-Styles - https://pandas.pydata.org/pandas-docs/stable/style.html
orange = sns.light_palette('orange', as_cmap=True)
pd.crosstab(chicago_crime.index_offense.fillna(' NaN'), chicago_crime.nonindex_offense.fillna(' NaN'), margins=True, margins_name='Total', dropna=False).style.background_gradient(cmap=orange)

nonindex_offense,0,1,Total
index_offense,,,
0,0,4110592,4110592
1,2765545,0,2765545
Total,2765545,4110592,6876137


In [31]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
print(chicago_crime.index_offense.value_counts(dropna=False))
round(chicago_crime.index_offense.value_counts(dropna=False) / len(chicago_crime), 3)

0    4110592
1    2765545
Name: index_offense, dtype: int64


0   0.598
1   0.402
Name: index_offense, dtype: float64

In [32]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
print(chicago_crime.nonindex_offense.value_counts(dropna=False))
round(chicago_crime.nonindex_offense.value_counts(dropna=False) / len(chicago_crime), 3)

1    4110592
0    2765545
Name: nonindex_offense, dtype: int64


1   0.598
0   0.402
Name: nonindex_offense, dtype: float64

Note: `description` is a secondary description of the IUCR code, a subcategory of the primary description. `primary_type`, the primary description, will be used instead for exploring data.

In [33]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
chicago_crime.description.value_counts(dropna=False)

SIMPLE                            810859
$500 AND UNDER                    557704
DOMESTIC BATTERY SIMPLE           528876
TO VEHICLE                        380963
TO PROPERTY                       361146
                                   ...  
SPORTS TAMPERING                       1
THEFT RETAIL                           1
SECOND DEGREE MURDER                   1
DOMESTIC VIOLENCE                      1
KEEP PLACE OF JUV PROSTITUTION         1
Name: description, Length: 374, dtype: int64

In [34]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
chicago_crime.location_description.value_counts(dropna=False)

STREET                   1798094
RESIDENCE                1164770
APARTMENT                 715533
SIDEWALK                  674237
OTHER                     261800
                          ...   
CLEANERS/LAUNDROMAT            1
PUBLIC GRAMMAR SCHOOL          1
POOLROOM                       1
LAGOON                         1
FUNERAL PARLOR                 1
Name: location_description, Length: 180, dtype: int64

In [35]:
value_counts = chicago_crime.location_description.value_counts(dropna=False)
df = value_counts.rename_axis('unique_values').reset_index(name='counts')
df['percent'] = round(df.counts / sum(df.counts), 3)

# Temporarily set options in the `with` statement
# Note: Settings are restored when you exit the 'with' block
with pd.option_context('max_rows', None, 'max_columns', None):
    print(df)

                                         unique_values   counts  percent
0                                               STREET  1798094    0.261
1                                            RESIDENCE  1164770    0.169
2                                            APARTMENT   715533    0.104
3                                             SIDEWALK   674237    0.098
4                                                OTHER   261800    0.038
5                       PARKING LOT/GARAGE(NON.RESID.)   197412    0.029
6                                                ALLEY   153088    0.022
7                             SCHOOL, PUBLIC, BUILDING   144339    0.021
8                                     RESIDENCE-GARAGE   133048    0.019
9                                   SMALL RETAIL STORE   122737    0.018
10                             RESIDENCE PORCH/HALLWAY   120341    0.018
11                              VEHICLE NON-COMMERCIAL   110651    0.016
12                                          RESTAUR

#### `location_description` values that were less than 1% of the distribution will be combined into the 'OTHER' group and a new variable will be creatd as `location_description_cat`.

In [36]:
# Return elements, either from `x` or `y`, depending on `condition`
gte_1pct = ['STREET', 'RESIDENCE', 'APARTMENT', 'SIDEWALK', 'OTHER', 'PARKING LOT/GARAGE(NON.RESID.)', 'ALLEY',
            'SCHOOL, PUBLIC, BUILDING', 'RESIDENCE-GARAGE', 'SMALL RETAIL STORE', 'RESIDENCE PORCH/HALLWAY',
            'VEHICLE NON-COMMERCIAL', 'RESTAURANT', 'GROCERY FOOD STORE', 'DEPARTMENT STORE', 'GAS STATION',
            'RESIDENTIAL YARD (FRONT/BACK)']
chicago_crime['location_description_gte_1pct'] = np.where(chicago_crime.location_description.isin(gte_1pct), 1, 0)

In [37]:
chicago_crime.location_description_gte_1pct.value_counts()

1    6025583
0     850554
Name: location_description_gte_1pct, dtype: int64

In [38]:
# Return elements, either from `x` or `y`, depending on `condition`
chicago_crime['location_description_cat'] = np.where(chicago_crime.location_description_gte_1pct == 0, 'OTHER', chicago_crime.location_description)
chicago_crime.drop('location_description_gte_1pct', axis=1, inplace=True)

In [39]:
value_counts = chicago_crime.location_description_cat.value_counts(dropna=False)
df = value_counts.rename_axis('unique_values').reset_index(name='counts')
df['percent'] = round(df.counts / sum(df.counts), 3)

# Temporarily set options in the `with` statement
# Note: Settings are restored when you exit the 'with' block
with pd.option_context('max_rows', None, 'max_columns', None):
    print(df)

                     unique_values   counts  percent
0                           STREET  1798094    0.261
1                        RESIDENCE  1164770    0.169
2                            OTHER  1112354    0.162
3                        APARTMENT   715533    0.104
4                         SIDEWALK   674237    0.098
5   PARKING LOT/GARAGE(NON.RESID.)   197412    0.029
6                            ALLEY   153088    0.022
7         SCHOOL, PUBLIC, BUILDING   144339    0.021
8                 RESIDENCE-GARAGE   133048    0.019
9               SMALL RETAIL STORE   122737    0.018
10         RESIDENCE PORCH/HALLWAY   120341    0.018
11          VEHICLE NON-COMMERCIAL   110651    0.016
12                      RESTAURANT   109257    0.016
13              GROCERY FOOD STORE    89016    0.013
14                DEPARTMENT STORE    86015    0.013
15                     GAS STATION    73797    0.011
16   RESIDENTIAL YARD (FRONT/BACK)    71448     0.01


In [40]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
chicago_crime.year.value_counts(dropna=False)

2002    486757
2001    485754
2003    475945
2004    469384
2005    453717
2006    448114
2007    437016
2008    427064
2009    392695
2010    370320
2011    351793
2012    335975
2013    307122
2014    275289
2016    268994
2017    268166
2018    266763
2015    264098
2019     91171
Name: year, dtype: int64

### Pivot Tables

In [41]:
# Return reshaped DataFrame organized by given index / column values
# Note: index: column to use to make new frame's index
#       columns: column to use to make new frame's column
#       values: column(s) to use for populating new frame's values; if not specified, all remaining columns will be
#               used and the result will have hierarchically indexed columns
#chicago_crime.pivot(index='year', columns='location_description_cat')

#### Pivot table `mean` using
- `pivot_table`
- `crosstab`
- `groupby`

#### `pivot_table`

In [42]:
# Create a spreadsheet-style pivot table as a DataFrame. The levels in the pivot table will be stored in MultiIndex
#    objects (hierarchical indexes) on the index and columns of the result DataFrame
# Note: values: column to aggregate, optional
#       index: keys to group by on the pivot table index
#       columns: keys to group by on the pivot table column
#       aggfunc: function, list of functions, dict, default numpy.mean
#       margins: add all row / columns (e.g. for subtotal / grand totals)
#       margins_name: name of the row / column that will contain the totals when margins is True; default 'All'
#       dropna: do not include columns whose entries are all NaN; default True
# Builtin-Styles - https://pandas.pydata.org/pandas-docs/stable/style.html
orange = sns.light_palette('orange', as_cmap=True)
round(chicago_crime.pivot_table(values='index_offense', index='location_description_cat', columns='year',
                                aggfunc='mean', margins=True, margins_name='All', dropna=False), 3).style.background_gradient(cmap=orange)

year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,All
location_description_cat,,,,,,,,,,,,,,,,,,,,
ALLEY,0.383,0.367,0.357,0.33,0.326,0.314,0.3,0.32,0.326,0.324,0.334,0.329,0.327,0.322,0.342,0.463,0.48,0.403,0.366,0.346
APARTMENT,0.323,0.325,0.308,0.311,0.323,0.307,0.308,0.325,0.343,0.352,0.37,0.374,0.342,0.321,0.306,0.321,0.316,0.317,0.306,0.328
DEPARTMENT STORE,0.858,0.857,0.876,0.86,0.862,0.837,0.835,0.87,0.898,0.89,0.891,0.876,0.856,0.841,0.827,0.799,0.791,0.792,0.777,0.85
GAS STATION,0.55,0.522,0.467,0.417,0.422,0.418,0.401,0.416,0.408,0.375,0.38,0.371,0.348,0.358,0.392,0.448,0.444,0.441,0.377,0.425
GROCERY FOOD STORE,0.738,0.745,0.715,0.722,0.715,0.707,0.692,0.721,0.751,0.727,0.743,0.748,0.726,0.738,0.746,0.726,0.751,0.706,0.713,0.727
OTHER,0.397,0.392,0.391,0.386,0.378,0.39,0.397,0.406,0.416,0.432,0.448,0.45,0.443,0.415,0.415,0.428,0.429,0.416,0.399,0.409
PARKING LOT/GARAGE(NON.RESID.),0.644,0.635,0.619,0.611,0.594,0.57,0.566,0.579,0.555,0.529,0.562,0.537,0.521,0.502,0.505,0.544,0.555,0.531,0.472,0.572
RESIDENCE,0.295,0.287,0.284,0.28,0.285,0.288,0.298,0.314,0.319,0.337,0.351,0.352,0.341,0.281,0.255,0.272,0.272,0.269,0.254,0.298
RESIDENCE PORCH/HALLWAY,0.34,0.321,0.297,0.294,0.285,0.285,0.306,0.309,0.303,0.312,0.321,0.319,0.342,0.374,0.412,0.457,0.505,0.561,0.566,0.343


#### `pivot_table` writing to CSV file

In [43]:
df = round(chicago_crime.pivot_table(values='index_offense', index='location_description_cat', columns='year',
                                     aggfunc='mean', margins=True, margins_name='All', dropna=False), 3)

# Create data path variable for writing data
data_path = '/Users/yangweichle/Documents/Employment/TRAINING/PYTHON/'

# Write object to a comma-separated values (csv) file
df.to_csv(data_path + 'pivot_table.csv', sep='\t', index=True, header=True)

#### `pivot_table` function

In [44]:
def pivot_table(df, values, index, columns, aggfunc, margins_name):
    # Create a spreadsheet-style pivot table as a DataFrame. The levels in the pivot table will be stored in MultiIndex
    #    objects (hierarchical indexes) on the index and columns of the result DataFrame
    # Note: values: column to aggregate, optional
    #       index: keys to group by on the pivot table index
    #       columns: keys to group by on the pivot table column
    #       aggfunc: function, list of functions, dict, default numpy.mean
    #       margins: add all row / columns (e.g. for subtotal / grand totals)
    #       margins_name: name of the row / column that will contain the totals when margins is True; default 'All'
    #       dropna: do not include columns whose entries are all NaN; default True
    # Builtin-Styles - https://pandas.pydata.org/pandas-docs/stable/style.html
    orange = sns.light_palette('orange', as_cmap=True)
    return round(df.pivot_table(values=values, index=index, columns=columns,
                                aggfunc=aggfunc, margins=True, margins_name=margins_name, dropna=False), 3).style.background_gradient(cmap=orange)

pivot_table(df=chicago_crime, values='index_offense', index='location_description_cat', columns='year', aggfunc='mean', margins_name='All')

year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,All
location_description_cat,,,,,,,,,,,,,,,,,,,,
ALLEY,0.383,0.367,0.357,0.33,0.326,0.314,0.3,0.32,0.326,0.324,0.334,0.329,0.327,0.322,0.342,0.463,0.48,0.403,0.366,0.346
APARTMENT,0.323,0.325,0.308,0.311,0.323,0.307,0.308,0.325,0.343,0.352,0.37,0.374,0.342,0.321,0.306,0.321,0.316,0.317,0.306,0.328
DEPARTMENT STORE,0.858,0.857,0.876,0.86,0.862,0.837,0.835,0.87,0.898,0.89,0.891,0.876,0.856,0.841,0.827,0.799,0.791,0.792,0.777,0.85
GAS STATION,0.55,0.522,0.467,0.417,0.422,0.418,0.401,0.416,0.408,0.375,0.38,0.371,0.348,0.358,0.392,0.448,0.444,0.441,0.377,0.425
GROCERY FOOD STORE,0.738,0.745,0.715,0.722,0.715,0.707,0.692,0.721,0.751,0.727,0.743,0.748,0.726,0.738,0.746,0.726,0.751,0.706,0.713,0.727
OTHER,0.397,0.392,0.391,0.386,0.378,0.39,0.397,0.406,0.416,0.432,0.448,0.45,0.443,0.415,0.415,0.428,0.429,0.416,0.399,0.409
PARKING LOT/GARAGE(NON.RESID.),0.644,0.635,0.619,0.611,0.594,0.57,0.566,0.579,0.555,0.529,0.562,0.537,0.521,0.502,0.505,0.544,0.555,0.531,0.472,0.572
RESIDENCE,0.295,0.287,0.284,0.28,0.285,0.288,0.298,0.314,0.319,0.337,0.351,0.352,0.341,0.281,0.255,0.272,0.272,0.269,0.254,0.298
RESIDENCE PORCH/HALLWAY,0.34,0.321,0.297,0.294,0.285,0.285,0.306,0.309,0.303,0.312,0.321,0.319,0.342,0.374,0.412,0.457,0.505,0.561,0.566,0.343


#### `crosstab`

In [45]:
# Compute a cross tabulation of two (or more) factors
# Note: index: values to group by in the rows
#       columns: values to group by in the columns
#       values: array of values to aggregate according to the factors; requires `aggfunc` be specified
#       aggfunc: function, optional; if specified, requires `values` be specified as well
#       margins: add row/column margins (subtotals)
#       margins_name: name of the row/column that will contain the totals when margins is True; default 'All'
#       dropna: do not include columns whose entries are all NaN; default True
# Builtin-Styles - https://pandas.pydata.org/pandas-docs/stable/style.html
orange = sns.light_palette('orange', as_cmap=True)
round(pd.crosstab(index=chicago_crime.location_description_cat.fillna(' NaN'), columns=chicago_crime.year.fillna(' NaN'), values=chicago_crime.index_offense,
                  aggfunc='mean', margins=True, margins_name='All', dropna=False), 3).style.background_gradient(cmap=orange)

year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,All
location_description_cat,,,,,,,,,,,,,,,,,,,,
ALLEY,0.383,0.367,0.357,0.33,0.326,0.314,0.3,0.32,0.326,0.324,0.334,0.329,0.327,0.322,0.342,0.463,0.48,0.403,0.366,0.346
APARTMENT,0.323,0.325,0.308,0.311,0.323,0.307,0.308,0.325,0.343,0.352,0.37,0.374,0.342,0.321,0.306,0.321,0.316,0.317,0.306,0.328
DEPARTMENT STORE,0.858,0.857,0.876,0.86,0.862,0.837,0.835,0.87,0.898,0.89,0.891,0.876,0.856,0.841,0.827,0.799,0.791,0.792,0.777,0.85
GAS STATION,0.55,0.522,0.467,0.417,0.422,0.418,0.401,0.416,0.408,0.375,0.38,0.371,0.348,0.358,0.392,0.448,0.444,0.441,0.377,0.425
GROCERY FOOD STORE,0.738,0.745,0.715,0.722,0.715,0.707,0.692,0.721,0.751,0.727,0.743,0.748,0.726,0.738,0.746,0.726,0.751,0.706,0.713,0.727
OTHER,0.397,0.392,0.391,0.386,0.378,0.39,0.397,0.406,0.416,0.432,0.448,0.45,0.443,0.415,0.415,0.428,0.429,0.416,0.399,0.409
PARKING LOT/GARAGE(NON.RESID.),0.644,0.635,0.619,0.611,0.594,0.57,0.566,0.579,0.555,0.529,0.562,0.537,0.521,0.502,0.505,0.544,0.555,0.531,0.472,0.572
RESIDENCE,0.295,0.287,0.284,0.28,0.285,0.288,0.298,0.314,0.319,0.337,0.351,0.352,0.341,0.281,0.255,0.272,0.272,0.269,0.254,0.298
RESIDENCE PORCH/HALLWAY,0.34,0.321,0.297,0.294,0.285,0.285,0.306,0.309,0.303,0.312,0.321,0.319,0.342,0.374,0.412,0.457,0.505,0.561,0.566,0.343


#### `groupby`

In [46]:
# Group DataFrame or Series using a mapper or by a Series of columns. A groupby operation involves some combination
#    of splitting the object, applying a function, and combining the results. This can be used to group large
#    amounts of data and compute operations on these groups.
# Note: by: used to determine the groups for the groupby
df = round(chicago_crime.groupby(by=['location_description_cat', 'year']).index_offense.mean(), 3)

# Temporarily set options in the `with` statement
# Note: Settings are restored when you exit the 'with' block
with pd.option_context('max_rows', None, 'max_columns', None):
    print(df)

location_description_cat        year
ALLEY                           2001   0.383
                                2002   0.367
                                2003   0.357
                                2004    0.33
                                2005   0.326
                                2006   0.314
                                2007     0.3
                                2008    0.32
                                2009   0.326
                                2010   0.324
                                2011   0.334
                                2012   0.329
                                2013   0.327
                                2014   0.322
                                2015   0.342
                                2016   0.463
                                2017    0.48
                                2018   0.403
                                2019   0.366
APARTMENT                       2001   0.323
                                2002   0.325
                  

#### Pivot table `sum` using
- `pivot_table`
- `crosstab`
- `groupby`

#### `pivot_table`

In [47]:
# Create a spreadsheet-style pivot table as a DataFrame. The levels in the pivot table will be stored in MultiIndex
#    objects (hierarchical indexes) on the index and columns of the result DataFrame
# Note: values: column to aggregate, optional
#       index: keys to group by on the pivot table index
#       columns: keys to group by on the pivot table column
#       aggfunc: function, list of functions, dict, default numpy.mean
#       margins: add all row / columns (e.g. for subtotal / grand totals)
#       margins_name: name of the row / column that will contain the totals when margins is True; default 'All'
#       dropna: do not include columns whose entries are all NaN; default True
# Builtin-Styles - https://pandas.pydata.org/pandas-docs/stable/style.html
orange = sns.light_palette('orange', as_cmap=True)
chicago_crime.pivot_table(values='index_offense', index='location_description_cat', columns='year',
                          aggfunc='sum', margins=True, margins_name='Total', dropna=False).style.background_gradient(cmap=orange)

year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Total
location_description_cat,,,,,,,,,,,,,,,,,,,,
ALLEY,4143,4003,3895,3606,3353,3293,3160,3138,2879,2759,2551,2554,2235,1951,1919,2454,2536,2009,595,53033
APARTMENT,9190,11630,12197,13146,13451,12956,13623,14924,15434,15022,15788,15457,13167,11300,10669,11024,10579,10930,3947,234434
DEPARTMENT STORE,5576,4781,4536,4063,3967,3791,3911,4532,4422,3964,3730,3636,3496,3361,3410,3355,3586,3740,1265,73122
GAS STATION,2495,2449,2284,2130,2229,2147,1985,1869,1568,1238,1300,1186,1088,1020,1172,1369,1724,1646,497,31396
GROCERY FOOD STORE,6349,5896,5200,4949,4540,3856,3569,3606,3498,2869,2747,2660,2369,2167,2199,2307,2634,2452,890,64757
OTHER,35244,32857,30239,29749,28011,27843,27239,26612,24050,23319,23400,23757,22206,18277,17330,19243,19774,19222,6368,454740
PARKING LOT/GARAGE(NON.RESID.),10384,9389,8771,8152,7303,6402,6427,6897,6253,5440,5954,5279,4448,3743,3751,4438,4581,4067,1283,112962
RESIDENCE,27795,25951,23133,21862,21607,21231,21835,22824,21067,20563,20452,18870,16795,12217,10624,12495,12450,11998,3860,347629
RESIDENCE PORCH/HALLWAY,3067,2896,2542,2404,2246,2185,2338,2271,2066,1874,1886,1874,1811,1820,1968,2084,2336,2635,926,41229


#### `crosstab`

In [48]:
# Compute a cross tabulation of two (or more) factors
# Note: index: values to group by in the rows
#       columns: values to group by in the columns
#       values: array of values to aggregate according to the factors; requires `aggfunc` be specified
#       aggfunc: function, optional; if specified, requires `values` be specified as well
#       margins: add row/column margins (subtotals)
#       margins_name: name of the row/column that will contain the totals when margins is True; default 'All'
#       dropna: do not include columns whose entries are all NaN; default True
# Builtin-Styles - https://pandas.pydata.org/pandas-docs/stable/style.html
orange = sns.light_palette('orange', as_cmap=True)
pd.crosstab(index=chicago_crime.location_description_cat.fillna(' NaN'), columns=chicago_crime.year.fillna(' NaN'), values=chicago_crime.index_offense,
            aggfunc='sum', margins=True, margins_name='Total', dropna=False).style.background_gradient(cmap=orange)

year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Total
location_description_cat,,,,,,,,,,,,,,,,,,,,
ALLEY,4143,4003,3895,3606,3353,3293,3160,3138,2879,2759,2551,2554,2235,1951,1919,2454,2536,2009,595,53033
APARTMENT,9190,11630,12197,13146,13451,12956,13623,14924,15434,15022,15788,15457,13167,11300,10669,11024,10579,10930,3947,234434
DEPARTMENT STORE,5576,4781,4536,4063,3967,3791,3911,4532,4422,3964,3730,3636,3496,3361,3410,3355,3586,3740,1265,73122
GAS STATION,2495,2449,2284,2130,2229,2147,1985,1869,1568,1238,1300,1186,1088,1020,1172,1369,1724,1646,497,31396
GROCERY FOOD STORE,6349,5896,5200,4949,4540,3856,3569,3606,3498,2869,2747,2660,2369,2167,2199,2307,2634,2452,890,64757
OTHER,35244,32857,30239,29749,28011,27843,27239,26612,24050,23319,23400,23757,22206,18277,17330,19243,19774,19222,6368,454740
PARKING LOT/GARAGE(NON.RESID.),10384,9389,8771,8152,7303,6402,6427,6897,6253,5440,5954,5279,4448,3743,3751,4438,4581,4067,1283,112962
RESIDENCE,27795,25951,23133,21862,21607,21231,21835,22824,21067,20563,20452,18870,16795,12217,10624,12495,12450,11998,3860,347629
RESIDENCE PORCH/HALLWAY,3067,2896,2542,2404,2246,2185,2338,2271,2066,1874,1886,1874,1811,1820,1968,2084,2336,2635,926,41229


#### `groupby`

In [49]:
# Group DataFrame or Series using a mapper or by a Series of columns. A groupby operation involves some combination
#    of splitting the object, applying a function, and combining the results. This can be used to group large
#    amounts of data and compute operations on these groups.
# Note: by: used to determine the groups for the groupby
df = chicago_crime.groupby(by=['location_description_cat', 'year']).index_offense.sum()

# Temporarily set options in the `with` statement
# Note: Settings are restored when you exit the 'with' block
with pd.option_context('max_rows', None, 'max_columns', None):
    print(df)

location_description_cat        year
ALLEY                           2001     4143
                                2002     4003
                                2003     3895
                                2004     3606
                                2005     3353
                                2006     3293
                                2007     3160
                                2008     3138
                                2009     2879
                                2010     2759
                                2011     2551
                                2012     2554
                                2013     2235
                                2014     1951
                                2015     1919
                                2016     2454
                                2017     2536
                                2018     2009
                                2019      595
APARTMENT                       2001     9190
                                2002    116

#### Pivot table `mean` using using Grouper function to aggregate based on date frequency (e.g., month end frequency)

In [50]:
# Convert a string to the datetime format
# Note: This is often slow -- consider doing it in the read_csv() method
chicago_crime['date'] = pd.to_datetime(chicago_crime['date']).dt.date
chicago_crime['date'] = pd.to_datetime(chicago_crime['date'])
type(chicago_crime.date[0])

pandas._libs.tslibs.timestamps.Timestamp

In [51]:
# Change the maximum number of rows and columns printed
# Note: 'None' value means unlimited
pd.set_option('max_rows', None)     # default: 60 rows
pd.set_option('max_columns', None)  # default: 20 columns

round(chicago_crime.pivot_table(values='index_offense', index=pd.Grouper(key='date', freq='M'), columns='location_description_cat',
                               aggfunc='mean'), 3)

# Reset the number of rows and columns printed to defaults
#pd.reset_option('max_rows')
#pd.reset_option('max_columns')

location_description_cat,ALLEY,APARTMENT,DEPARTMENT STORE,GAS STATION,GROCERY FOOD STORE,OTHER,PARKING LOT/GARAGE(NON.RESID.),RESIDENCE,RESIDENCE PORCH/HALLWAY,RESIDENCE-GARAGE,RESIDENTIAL YARD (FRONT/BACK),RESTAURANT,"SCHOOL, PUBLIC, BUILDING",SIDEWALK,SMALL RETAIL STORE,STREET,VEHICLE NON-COMMERCIAL
date,,,,,,,,,,,,,,,,,
2001-01-31,0.403,0.313,0.861,0.541,0.748,0.383,0.69,0.276,0.293,0.741,nan,0.49,0.211,0.231,0.681,0.424,0.429
2001-02-28,0.354,0.315,0.877,0.578,0.727,0.376,0.67,0.277,0.284,0.682,nan,0.462,0.208,0.209,0.674,0.421,0.417
2001-03-31,0.345,0.307,0.857,0.564,0.737,0.37,0.627,0.264,0.289,0.662,nan,0.502,0.166,0.25,0.699,0.412,0.446
2001-04-30,0.356,0.286,0.857,0.574,0.724,0.379,0.621,0.283,0.321,0.66,nan,0.479,0.213,0.275,0.664,0.422,0.434
2001-05-31,0.39,0.32,0.856,0.501,0.728,0.394,0.638,0.292,0.352,0.716,nan,0.496,0.209,0.273,0.683,0.424,0.452
2001-06-30,0.375,0.316,0.864,0.542,0.726,0.405,0.611,0.296,0.355,0.767,nan,0.477,0.309,0.278,0.727,0.44,0.495
2001-07-31,0.379,0.332,0.856,0.591,0.721,0.414,0.652,0.303,0.405,0.793,0.0,0.5,0.337,0.298,0.691,0.459,0.529
2001-08-31,0.39,0.323,0.871,0.554,0.711,0.412,0.637,0.307,0.354,0.75,nan,0.538,0.453,0.292,0.707,0.459,0.518
2001-09-30,0.389,0.332,0.839,0.521,0.769,0.413,0.646,0.309,0.39,0.733,0.0,0.548,0.223,0.295,0.717,0.476,0.535


In [52]:
# Change the maximum number of rows and columns printed
# Note: 'None' value means unlimited
pd.set_option('max_rows', None)     # default: 60 rows
pd.set_option('max_columns', None)  # default: 20 columns

round(chicago_crime.pivot_table(values='index_offense', index=pd.Grouper(key='date', freq='M'), columns='location_description_cat',
                               aggfunc='sum'), 3)

# Reset the number of rows and columns printed to defaults
#pd.reset_option('max_rows')
#pd.reset_option('max_columns')

location_description_cat,ALLEY,APARTMENT,DEPARTMENT STORE,GAS STATION,GROCERY FOOD STORE,OTHER,PARKING LOT/GARAGE(NON.RESID.),RESIDENCE,RESIDENCE PORCH/HALLWAY,RESIDENCE-GARAGE,RESIDENTIAL YARD (FRONT/BACK),RESTAURANT,"SCHOOL, PUBLIC, BUILDING",SIDEWALK,SMALL RETAIL STORE,STREET,VEHICLE NON-COMMERCIAL
date,,,,,,,,,,,,,,,,,
2001-01-31,193.0,685.0,513.0,179.0,582.0,"2,797.0",890.0,"2,248.0",179.0,541.0,nan,302.0,181.0,394.0,431.0,"4,923.0",102.0
2001-02-28,196.0,610.0,422.0,175.0,437.0,"2,463.0",744.0,"1,961.0",163.0,367.0,nan,235.0,216.0,355.0,357.0,"4,236.0",101.0
2001-03-31,268.0,740.0,449.0,186.0,516.0,"2,903.0",876.0,"2,129.0",181.0,448.0,nan,326.0,256.0,508.0,402.0,"5,000.0",112.0
2001-04-30,343.0,700.0,418.0,205.0,479.0,"2,860.0",771.0,"2,229.0",238.0,529.0,nan,256.0,179.0,652.0,421.0,"5,204.0",109.0
2001-05-31,397.0,743.0,468.0,211.0,526.0,"3,045.0",817.0,"2,302.0",294.0,634.0,nan,277.0,280.0,717.0,448.0,"5,396.0",117.0
2001-06-30,381.0,733.0,433.0,205.0,460.0,"3,106.0",808.0,"2,457.0",295.0,679.0,nan,259.0,142.0,767.0,461.0,"5,795.0",147.0
2001-07-31,448.0,815.0,481.0,250.0,517.0,"3,327.0",951.0,"2,570.0",384.0,883.0,0.0,259.0,93.0,913.0,432.0,"6,668.0",157.0
2001-08-31,438.0,809.0,512.0,253.0,525.0,"3,301.0",991.0,"2,578.0",320.0,723.0,nan,310.0,68.0,867.0,446.0,"6,481.0",171.0
2001-09-30,396.0,785.0,402.0,231.0,617.0,"3,032.0",885.0,"2,340.0",340.0,666.0,0.0,323.0,191.0,788.0,463.0,"6,247.0",229.0


#### VLookup using Pandas

In [53]:
import pandas as pd

In [54]:
df1 = pd.DataFrame({'Security': ['ABC', 'DEF', 'JHK', 'LMN', 'OPQ'],
                    'ISIN': ['I1', 'I2', 'I3', 'I4', 'I5']})
df1

,Security,ISIN
0,ABC,I1
1,DEF,I2
2,JHK,I3
3,LMN,I4
4,OPQ,I5


In [55]:
df2 = pd.DataFrame({'Value': [100, 200, 300],
                    'ISIN': ['I2', 'I3', 'I5']})
df2

,Value,ISIN
0,100,I2
1,200,I3
2,300,I5


#### `left join`

In [56]:
# Merge DataFrame or named Series objects with a database-style join
# Note: how: type of merge to be performed; default 'inner'
#            * left: use only keys from left frame, similar to a SQL left outer join; preserve key order.
#            * right: use only keys from right frame, similar to a SQL right outer join; preserve key order
#            * outer: use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically
#            * inner: use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys
#       on: column or index level names to join on; these must be found in both DataFrames
left = pd.merge(df1, df2, how='left', on='ISIN')
left

,Security,ISIN,Value
0,ABC,I1,nan
1,DEF,I2,100.0
2,JHK,I3,200.0
3,LMN,I4,nan
4,OPQ,I5,300.0


In [57]:
pd.merge(df1, df2, how='left', on='ISIN')[['Security', 'Value']].replace(np.nan, 0)

,Security,Value
0,ABC,0.0
1,DEF,100.0
2,JHK,200.0
3,LMN,0.0
4,OPQ,300.0


#### `right join`

In [58]:
right = pd.merge(df1, df2, how='right', on='ISIN')
right

,Security,ISIN,Value
0,DEF,I2,100
1,JHK,I3,200
2,OPQ,I5,300


In [59]:
pd.merge(df1, df2, how='right', on='ISIN')[['Security', 'Value']].replace(np.nan, 0)

,Security,Value
0,DEF,100
1,JHK,200
2,OPQ,300


#### `inner join`

In [60]:
inner = pd.merge(df1, df2, how='inner', on='ISIN')
inner

,Security,ISIN,Value
0,DEF,I2,100
1,JHK,I3,200
2,OPQ,I5,300


In [61]:
pd.merge(df1, df2, how='inner', on='ISIN')[['Security', 'Value']].replace(np.nan, 0)

,Security,Value
0,DEF,100
1,JHK,200
2,OPQ,300
